# Use an existing Bing Agent with Semantic Kernel
This notebook demonstrates how to create a Bing Agent using Semantic Kernel. It includes a custom monkey-patch for citation URL extraction and shows how to simulate a conversation with an existing Azure AI Agent.

## Prerequisties

#### Complete the agent setup and create a project connection to a Bing  resource

1. Create an agent project by deploying a template in the [quickstart](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=programming-language-python-azure).

2. Create a Grounding with Bing Search resource and create a project connection to this resource. [Learn more](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview)
    1. You need to have `Owner` or `Contributor` role in your subscription or resource group to create it.

#### Install the required libraries  
In this section, we import all the libraries we need. The code imports asynchronous Azure Identity for authentication and the necessary Semantic Kernel modules to work with Azure AI Agents.

In [29]:
%pip install azure-ai-projects
%pip install azure-identity
%pip install semantic-kernel[azure]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Step 1: Import required libraries 


In [30]:
import asyncio

from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings, agent_content_generation
from semantic_kernel.contents import AuthorRole

## Step 2: Annotation path for Bing urls
Here, we override the default annotation processing function to capture citation URLs when they're returned by the agent. The new function checks if the annotation has the required attributes (start_index and end_index) and, if it’s a URL citation, extracts and prints the Bing Search URL. The semantic kernel Agent SDK only supports file search annotations currently.

In [31]:
# Global variable to store citation URLs
citation_url = "No citations available"

# Save the original function
original_generate_annotation_content = agent_content_generation.generate_annotation_content
def patched_generate_annotation_content(annotation):
    global citation_url
    print("Patched function called")  # Debug print
    
    # Ensure that the annotation has both 'start_index' and 'end_index' attributes.
    # print(f"# Annotation before: '{annotation}'")
    if not hasattr(annotation, "start_index"):
        setattr(annotation, "start_index", None)
        # print(f"# Annotation set start_index: '{annotation}'")
    if not hasattr(annotation, "end_index"):
        setattr(annotation, "end_index", None)
        # print(f"# Annotation set end_index: '{annotation}'")

    # print(f"# Annotation after: '{annotation}'")
    # Check if this is a URL citation annotation and expose the URL
    if hasattr(annotation, 'type') and annotation.type == 'url_citation':
        try:
            citation_url = annotation['url_citation']['url']
            # print(f"# Citation url: '{citation_url}'")
            return annotation
        except Exception as e:
            print(f"Error processing citation: {e}")
    # Return the annotation as is for other annotation types
    return annotation

# Apply the monkey-patch
agent_content_generation.generate_annotation_content = patched_generate_annotation_content

print("Bing citation patch function run")  # Debug print

Bing citation patch function run


## Step 3: Conversation Setup 

1. **Retrieve Bing Connection and Initialize Tool:**
 
    First, the program retrieves your project's Bing connection and initializes the Grounding with Bing Search tool.

1. **Define an Agent with the Bing Tool Enabled**

    Next, an agent definition is created using Azure AI Agent Service, and the Bing Search tool is attached.

1. **Initialize Semantic Kernel Agent:**

    Then, the semantic kernel agent is initialized using the defintion.

1.  **Stimulate a Conversation:**
    
    Finally, a conversation thread is created. A user message is sent to the agent, which generates a response. The agent's reply is displayed along with any citation URLs captured by the patched annotation function.


In [32]:
# Simulate a conversation with the agent
USER_INPUTS = [
    "What is the weather in seattle?",
]

In [33]:
ai_agent_settings = AzureAIAgentSettings.create()
async def main() -> None:
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        
         # Get your Bing connection        
        bing_connection = await client.connections.get(
                connection_name="decdemobing", include_credentials=True  # Optional. Defaults to "False"
        )
        conn_id = bing_connection.id
        print(conn_id)

        # Initialize the Bing Search tool
        bing = BingGroundingTool(connection_id=conn_id)
        
        # 1. Create an agent using Azure AI agent service
        agent_definition = await client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name="Bing Agent",
            instructions="Answer the user's questions.",
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"},
        )
        print("Created agent definition")
        
        agent_definition = await client.agents.get_agent(
            assistant_id="asst_dD17QuoyrPo14guVyzKAJkX2",
        )

        # 2. Create a Semantic Kernel agent for the Azure AI agent
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )

        # 3. Create a new thread on the Azure AI agent service
        thread = await client.agents.create_thread()
        try:
            for user_input in USER_INPUTS:
                
                # 4. Add the user input as a chat message
                await agent.add_chat_message(thread_id=thread.id, message=user_input)
                print(f"# User: '{user_input}'")


                # 5. Invoke the agent for the specified thread for response
                response = await agent.get_response(thread_id=thread.id)
                print(f"# {response.name}: {response}")

                 # Retrieve run step details to get Bing Search query link
                 # This code is not working so it is commented out
                # runs = await client.agents.list_runs(thread_id=thread.id)
                # print(f"# The most recent run: '{runs}'")

                # run_id = runs['last_id']
                # print(f"# The most recent run_id: '{run_id}'")


                # run_steps = await client.agents.list_run_steps(run_id = run_id, thread_id=thread.id)
                # for run_step in run_steps['data']:
                #     if 'bing_search_query' in run_step:
                #         search_query = run_step['bing_search_query']
                #         bing_search_url = f"https://www.bing.com/search?q={search_query.replace(' ', '+')}"
                #         print(f"Bing Search URL: {bing_search_url}")
                # Print the captured citation URL
                print(f"Citation URL: {citation_url}")


        finally:
            # 6. Cleanup: Delete the thread and agent
            await client.agents.delete_thread(thread.id)
            # Do not clean up the assistant so it can be used again

In [34]:
await main()


/subscriptions/921496dc-987f-410f-bd57-426eb2611356/resourceGroups/rg-amanda-standard/providers/Microsoft.MachineLearningServices/workspaces/project-demo-dyl6/connections/decdemobing
Created agent definition
# User: 'What is the weather in seattle?'
Patched function called
# BingAgent: The weather in Seattle today, March 10, 2025, is cloudy with a high temperature around 50°F and a low around 40°F. Winds are expected to be from the south at 5 to 10 mph【3†source】.
Citation URL: https://world-weather.info/forecast/usa/seattle/march-2025/
